STA 141B Project - Impact of Social Media interactions with Spotify popularity

1. Introduction
2. Data Acquisiton 
3. Data Process 
4. Approach
5. Data Analysis 
5b. General statistics 
5c. Data Analysis real 
6. Reports + visualization
7. Dicussion 
8. 

In [2]:
print("hi")

hi
